<a href="https://colab.research.google.com/github/kg4-ken1ro/mypandas_tutorial_7/blob/main/study_tutorial_7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%matplotlib inline
import warnings
import numpy as np
import pandas as pd
import xgboost as xgb
import lightgbm as lgb
import matplotlib.pylab as plt

from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import ParameterGrid

warnings.filterwarnings('ignore')

In [3]:
all_params = {
    'max_depth': [3, 5, 7],
    'learning_rate': [0.1],
    'min_child_weight': [3, 5, 10],
    'n_estimetors': [10000],
    'colsample_bytree': [0.8, 0.9, 1.0],
    'colsample_bylevel': [0.8, 0.9, 1.0],
    'reg_alpha': [0, 0.1],
    'random_state': [0],
    'n_jobs': [1],
}

for params in ParameterGrid(all_params):
    print(params)
    break 

{'colsample_bylevel': 0.8, 'colsample_bytree': 0.8, 'learning_rate': 0.1, 'max_depth': 3, 'min_child_weight': 3, 'n_estimetors': 10000, 'n_jobs': 1, 'random_state': 0, 'reg_alpha': 0}


In [4]:
def preprocess_df(df):
    # CabinはこのあとDropするので、コードから削除
    df["Age"] = df["Age"].fillna(df["Age"].mean())
    df["Embarked"] = df["Embarked"].fillna(df["Embarked"].mode())
    df["FamilySize"] = df["SibSp"] + df["Parch"] + 1
   
    # 列の削除
    df.drop(["Name", "Ticket", "Cabin", "PassengerId"], axis=1, inplace=True)

    # Sexの01化とEmbarkedのダミー化 
    df["Sex"] = df["Sex"].replace({"male": 0, "female": 1})
    df = pd.get_dummies(df)

    return df

In [5]:

# main
df_train = pd.read_csv("drive/MyDrive/train.csv")
train_y = df_train["Survived"]
train_x = df_train.drop("Survived", axis=1)

train_x = preprocess_df(train_x)

accuracies = []

cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=0)
for train_idx, test_idx in cv.split(train_x, train_y):
  trn_x = train_x.iloc[train_idx, :]
  val_x = train_x.iloc[test_idx, :]

  trn_y = train_y.iloc[train_idx]
  val_y = train_y.iloc[test_idx]

  clf_xgb = xgb.XGBClassifier(**params)
  clf_lgb = lgb.LGBMClassifier(**params)

  clf_xgb.fit(trn_x, trn_y)
  clf_lgb.fit(trn_x, trn_y)

  # 平均値化するためにprobabilityを出す
  pred_proba_y_xgb = clf_xgb.predict_proba(val_x)[:, 1]
  pred_proba_y_lgb = clf_lgb.predict_proba(val_x)[:, 1]

  # probabilityの平均値が0.50を超えていれば1, そうでな場合0
  pred_proba_y = pd.DataFrame({"xgb": pred_proba_y_xgb, "lgb": pred_proba_y_lgb}).mean(axis=1)
  pred_y = [1 if proba > 0.50 else 0 for proba in pred_proba_y]
  accuracies.append(accuracy_score(val_y, pred_y))

print(np.mean(accuracies))

[23:19:39] WARNING: ../src/learner.cc:767: 
Parameters: { "n_estimetors" } are not used.

[LightGBM] [Warning] Unknown parameter: colsample_bylevel
[LightGBM] [Warning] Unknown parameter: n_estimetors
[23:19:39] WARNING: ../src/learner.cc:767: 
Parameters: { "n_estimetors" } are not used.

[LightGBM] [Warning] Unknown parameter: colsample_bylevel
[LightGBM] [Warning] Unknown parameter: n_estimetors
[23:19:39] WARNING: ../src/learner.cc:767: 
Parameters: { "n_estimetors" } are not used.

[LightGBM] [Warning] Unknown parameter: colsample_bylevel
[LightGBM] [Warning] Unknown parameter: n_estimetors
0.8204264870931537
